In [ ]:
import pandas as pd
import warnings
import requests, zipfile, io, os
import numpy as np
from tensorflow import keras
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
warnings.filterwarnings('ignore')

In [ ]:
# download file from url, extract it and store it in the dest_folder
def download_and_extract(url, dest_folder):
  r = requests.get(url)
  z = zipfile.ZipFile(io.BytesIO(r.content))
  z.extractall(dest_folder)

In [ ]:
# define folders and urls
healthy_path = './healthy'
healthy_url = 'http://wwwp.fc.unesp.br/~papa/pub/datasets/Handpd/NewHealthy/Signal.zip'
parkinson_path = './parkinson'
parkinson_url = 'http://wwwp.fc.unesp.br/~papa/pub/datasets/Handpd/NewPatients/Signal.zip'

In [ ]:
download_and_extract(healthy_url, healthy_path)

In [ ]:
download_and_extract(parkinson_url, parkinson_path)

In [ ]:
# actual downloaded signals
healthy_signals = f'{healthy_path}/Signal/'
parkinson_signals = f'{parkinson_path}/Signal/'

In [ ]:
# define Input and labels (0 = healthy, parkinson = 1)
X = []
y = []

In [ ]:
# create dataset with sequence length 500

# healthy signals
for filename in os.listdir(healthy_signals):
  if filename.endswith('txt'):
    df = pd.read_csv(healthy_signals + filename, sep='\s+', header=None, comment='#', nrows=250) 
    X.append(df.to_numpy())
    y.append(0)

# parkinson signals
for filename in os.listdir(parkinson_signals):
  if filename.endswith('txt'):
    df = pd.read_csv(parkinson_signals + filename, sep='\s+', header=None, comment='#', nrows=250) 
    X.append(df.to_numpy())
    y.append(1)


In [ ]:
# convert lists to np arrays
X = np.array(X)
y = np.array(y)

In [ ]:
X.shape

(792, 250, 6)

In [ ]:
y.shape

(792,)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
model = keras.models.Sequential([
keras.layers.LSTM(128, return_sequences=True,  input_shape=(X_train.shape[1], 6)),
keras.layers.LSTM(128),
keras.layers.Dense(1, activation="sigmoid")
])


In [ ]:
model.compile(loss="binary_crossentropy", optimizer="adam",
metrics=["accuracy"])


In [ ]:
model.fit(X_train, y_train, epochs = 10, batch_size = 32)

Epoch 1/10
17/17 [==============================] - 0s 24ms/step - loss: 0.6551 - accuracy: 0.6226
Epoch 2/10
17/17 [==============================] - 0s 23ms/step - loss: 0.6332 - accuracy: 0.6283
Epoch 3/10
17/17 [==============================] - 0s 23ms/step - loss: 0.6625 - accuracy: 0.5943
Epoch 4/10
17/17 [==============================] - 0s 23ms/step - loss: 0.6416 - accuracy: 0.6302
Epoch 5/10
17/17 [==============================] - 0s 23ms/step - loss: 0.6361 - accuracy: 0.6377
Epoch 6/10
17/17 [==============================] - 0s 23ms/step - loss: 0.6743 - accuracy: 0.5943
Epoch 7/10
17/17 [==============================] - 0s 23ms/step - loss: 0.6555 - accuracy: 0.6453
Epoch 8/10
17/17 [==============================] - 0s 23ms/step - loss: 0.6573 - accuracy: 0.6264
Epoch 9/10
17/17 [==============================] - 0s 23ms/step - loss: 0.6393 - accuracy: 0.6472
Epoch 10/10
17/17 [==============================] - 0s 23ms/step - loss: 0.6299 - accuracy: 0.6396


In [ ]:
predictions = model.predict(X_test,verbose=0)

In [ ]:
print(model.summary())

In [ ]:
plt.plot(X_train, color = 'black', label = 'train')
plt.plot(X_test, color = 'green', label = 'test')
plt.title('Prediction')
plt.xlabel('A')
plt.ylabel('B')
plt.legend()
plt.show()

In [ ]:
model.evaluate(X_test, y_test)


9/9 [==============================] - 0s 10ms/step - loss: 0.6469 - accuracy: 0.6069


[0.6469305157661438, 0.6068702340126038]